# Задание 1
## Формулировка
Добро пожаловать в 2912 год, где ваши навыки работы с данными понадобятся для разгадки космической тайны. Мы получили передачу с расстояния в четыре световых года, и дела обстоят не очень хорошо.

Космический корабль "Титаник" был межзвездным пассажирским лайнером, запущенным месяц назад. Cудно отправилось в свой первый рейс, перевозя эмигрантов из нашей Солнечной системы на три недавно пригодные для жизни экзопланеты, вращающиеся вокруг близлежащих звезд.

Огибая Альфу Центавра по пути к своему первому пункту назначения — знойной 55 Cancri E — неосторожный космический корабль "Титаник" столкнулся с пространственно-временной аномалией, скрытой в пылевом облаке. К сожалению, его постигла та же участь, что и его тезку 1000 лет назад. Хотя корабль остался нетронутым, почти половина пассажиров была перенесена в альтернативное измерение!

Ваша задача - предсказать, был ли пассажир перенесен в альтернативное измерение во время столкновения космического корабля "Титаник" с пространственно-временной аномалией. Чтобы помочь вам сделать эти прогнозы, вам предоставляется набор личных записей, восстановленных из поврежденной компьютерной системы корабля.

Подготовьте данные для обучения, составьте любую понравившуюся вам линейную модель, и решите задачу бинарной классификации.  
## Описание данных
В папке `data` вы ныйдете фаил `assignment_1_data.csv` в котором будут содержаться следующие поля:
 - `PassengerId` - уникальный идентификатор для каждого пассажира. Каждый идентификатор имеет форму gggg_pp, где gggg указывает группу, с которой путешествует пассажир, а pp - его номер в группе. Люди в группе часто являются членами семьи, но не всегда.
 - `HomePlanet` - планета, с которой вылетел пассажир, обычно планета его постоянного проживания.
 - `CryoSleep` - указывает, решил ли пассажир погрузиться в анабиоз на время путешествия. Пассажиры, находящиеся в криосне, прикованы к своим каютам.
 - `Cabin` - номер каюты, в которой находится пассажир. Принимает форму deck/num/side, где side может быть либо P для левого, либо S для правого борта.
 - `Destination` - планета, на которой пассажир будет высаживаться.
 - `Age` - возраст пассажира.
 - `VIP` - Оплатил ли пассажир специальное VIP-обслуживание во время рейса.
 - `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` - суммы, которую пассажир оплатил за пользование всеми многочисленными роскошными удобствами космического корабля "Титаник".
 - `Name` - имя и фамилия пассажира.
 - `Transported` - был ли пассажир перенесен в другое измерение. Это цель, столбец, который вы пытаетесь предсказать.

## Уточнение
Для решения этой задачи подойдет любая линейная модель, информацию о которых вы можете подчерпнуть из лекций и семинаров в папке `materials`. Решение оформите в юпитер ноутбуке (желательно). Оно должно состоять из нескольких шагов:

1. Необходимо проанализировать и подготовить данные, сформировать тренировочную и тестовую выборки.
2. Выбрать и обучить лнейную модель.
3. Оценить ее качество, посчитав необходимые метрики.

Каждый шаг должен сопровождаться комментириями/пояснениями (очень развернуто не надо). Проинтерпритируйте получившийся результат. 

# Импортируем библиотеки

In [964]:
import random

import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

from sklearn.tree import DecisionTreeClassifier

In [965]:
RANDOM_STATE = 42
TEST_SIZE = 0.3

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# Анализ данных

In [966]:
# Выгружаем данные, и смотрим на первые 5, и последние 5 строк в DataFrame

df = pd.read_csv("../data/assignment_1.csv")

df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [967]:
# Смотрим на статистики по вещественным признакам.
# Можно заметить, что строка count не равна числу строк в df.
# Это говорит нам о наличие пропусков

# у 5 из 6 признаков 50-ый персентиль равен 0

df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [968]:
# Посмотрим на пропуски в данных

df.isna().sum()


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

# Подготовка данных

In [969]:
# Для удобства организуем наши признаки

numerical_features = [
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "VIP",
    "CryoSleep",
]


cat_features = [
    "HomePlanet",
    "Destination"
]

drop_features = [
    "PassengerId",
    "Cabin",
    "Name"
]

target = "Transported"

all_features = numerical_features + cat_features

In [970]:
# Заполним пропуски самым часто встречающимся значением

all_features = numerical_features + cat_features

for feat in all_features:
    num_mode = df[feat].mode().values[0]

    df[feat] = df[feat].fillna(value=num_mode)


In [971]:
# Выбросим признаки, которые являются категориальными, но из-за высокой вариативности нам не подходят

df = df.drop(columns=drop_features)


In [972]:
# Избавились от пропусков!

df.isna().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [973]:
# Приводим целевую переменную к числовому значению
# В нашем случае это бинарная классификация, целевое признак может принимать всего два значения

df[target] = df[target].astype(int)
df["CryoSleep"] = df["CryoSleep"].astype(int)
df["VIP"] = df["VIP"].astype(int)


In [974]:
# One-hot encoding для признака HomePlanet, Destination

df = pd.concat((df, pd.get_dummies(df[["HomePlanet", "Destination"]])), axis=1)


In [975]:
# Выбрасываем старые признаки

df = df.drop(columns=["HomePlanet", "Destination"])


In [976]:
all_features = [feature for feature in df.columns if feature != target]


# Делим на train/test

In [977]:
X = df[all_features]
y = df[target]


In [978]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE)


# Линейная модель

In [979]:
lr_pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(random_state=RANDOM_STATE))
    ]
)


In [980]:
lr_pipe.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('lr', LogisticRegression(random_state=42))])

## Посмотрим на метрики

In [981]:
y_pred = lr_pipe.predict_proba(X_test)[:, 1]
y_pred_bin = (y_pred > 0.5).astype(int)

print(f"ROC-AUC score: {roc_auc_score(y_test, y_pred)}")


ROC-AUC score: 0.8641546743865838


In [982]:
print(classification_report(y_test, y_pred_bin))


              precision    recall  f1-score   support

           0       0.79      0.75      0.77      1289
           1       0.77      0.81      0.79      1319

    accuracy                           0.78      2608
   macro avg       0.78      0.78      0.78      2608
weighted avg       0.78      0.78      0.78      2608



# Нелинейная модель

В качестве нелинейной модели воспользуюсь градиентном бустингом

In [983]:
decision_tree = DecisionTreeClassifier(
    max_depth=6,
    splitter="best",
    min_samples_leaf=50, 
    criterion="gini", 
    random_state=RANDOM_STATE)

decision_tree.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=6, min_samples_leaf=50, random_state=42)

In [984]:
y_pred = decision_tree.predict_proba(X_test)[:, 1]
y_pred_bin = (y_pred > 0.5).astype(int)


## Посмотрим на метрики

In [985]:
print(f"ROC-AUC score: {roc_auc_score(y_test, y_pred)}")

ROC-AUC score: 0.8499559755345135


In [986]:
print(classification_report(y_test, y_pred_bin))


              precision    recall  f1-score   support

           0       0.85      0.64      0.73      1289
           1       0.72      0.89      0.79      1319

    accuracy                           0.76      2608
   macro avg       0.78      0.76      0.76      2608
weighted avg       0.78      0.76      0.76      2608



# Вывод

Данные неплохо разделяются как линейными моделями, так и нелинейными.  
Разницу можем лишь заметить в метриках precision, recall.
